<hr style="border:2px solid gray"> </hr>

# Homework 1 - Traveling Salesman Problem

## Example Code

### Algorithm 4: Genetic Algorithm 

### Author: Wangduk Seo (CAU AI Lab)
<hr style="border:2px solid gray"> </hr>

# Step 0. Importing packages and Global Settings

---------------------------------------------------------------
## (Optional) For Colab

In [ ]:
# from google.colab import drive
# import os, sys
# drive.mount('gdrive', force_remount=True)

---------------------------------------------------------------

In [ ]:
# package list
import numpy as np
import sys
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.pyplot as plt
import time

# Global Variables
# GA
POOL_SIZE = 4 
TOURNAMENT_SIZE = 3
MAX_ITERATION = 1000

# SA
MAX_EVALUATION = 10
SUB_ITERATIONS = 5
TEMPERATURE = 100
COOLING_RATIO = 0.4
TEMP_LIMIT = 1

np.random.seed(0)
# Plot Settings
PLOT_MODE = True # Draw Route
plt.ion()

# First City Index
FIRST_IDX = 0

In [ ]:
file_path = 'data1.txt'

# Step 1. Data Loading

In [ ]:
def fileloader():
    #     Data Format
    #     ---------------------------------------------------------
    #     NAME : pia3056
    #     COMMENT : Bonn VLSI data set with 3056 points
    #     COMMENT : Uni Bonn, Research Institute for Discrete Math
    #     COMMENT : Contributed by Andre Rohe
    #     TYPE : TSP
    #     DIMENSION : 3056 -----------------------------|
    #     EDGE_WEIGHT_TYPE : EUC_2D                     |
    #     NODE_COORD_SECTION                            |
    #     1 0 11 (2 dimentional coordinate of city)     |
    #     2 0 115                                       |
    #     ...                                           |
    #     ...(Total 3056 nodes)<------------------------|
    #     EOF
    #     ---------------------------------------------------------
    with open(file_path, "r") as file:
        file_str = file.readlines()

    # Get the coordinates of cities
    coord_str = file_str[8:-1]  # first city string to last city string (EOF 전까지)
    coord_list = np.zeros((len(coord_str), 2))
    for idx, item in enumerate(coord_str):
        items = item.split()
        coord_list[idx, 0], coord_list[idx, 1] = float(items[1]), float(items[2])

    return coord_list

# Step 2. Initialization

In [ ]:
def initialize_greedy(coord_list, first_idx):
    cnt_cities = len(coord_list)
    # Initialize path and insert first city index to the first and last elements
    path = np.zeros(cnt_cities + 1, dtype=np.int32)
    path[0], path[-1] = first_idx, first_idx

    # Euclidean distance map between cities
    path_map = euclidean_distances(coord_list, coord_list)

    cities_tovisit = np.ones((cnt_cities), dtype=np.bool_)
    cities_tovisit[first_idx] = False

    # Iteratively Connect nearest cities
    for i in range(1, cnt_cities):
        start_idx = path[i - 1]
        distance_from_start = path_map[start_idx, :]
        nearest_list = np.argsort(distance_from_start)
        for idx in range(len(nearest_list)):
            # check the nearest city is visited
            if cities_tovisit[nearest_list[idx]]:
                nearest_city = nearest_list[idx]
                break
        cities_tovisit[nearest_city] = False
        path[i] = nearest_city

    return path_map, path

def initialize_greedy_improve(coord_list, first_idx):
    cnt_cities = len(coord_list)
    # Initialize path and insert first city index to the first and last elements
    path = np.zeros(cnt_cities + 1, dtype=np.int32)


    # Euclidean distance map between cities
    path_map = euclidean_distances(coord_list, coord_list)

    cities_tovisit = np.ones((cnt_cities), dtype=np.bool_)
    

    path_map_copy=path_map.copy()
    longest_list = np.zeros(cnt_cities, dtype=np.int32)
    for i in range(0,cnt_cities):
        longest=np.argsort(-path_map_copy[i])
        longest_list[i]=path_map_copy[i][longest[0]]
    longest_list.sort()


    path[0], path[-1] = longest_list[0], longest_list[0]
    cities_tovisit[longest_list[0]] = False
    # Iteratively Connect nearest cities
    for i in range(1, cnt_cities):
        start_idx = path[i - 1]
        distance_from_start = path_map[start_idx, :]
        nearest_list = np.argsort(distance_from_start)
        for idx in range(len(nearest_list)):
            # check the nearest city is visited
            if cities_tovisit[nearest_list[idx]]:
                nearest_city = nearest_list[idx]
                break
        cities_tovisit[nearest_city] = False
        path[i] = nearest_city
    checker=0
    idx_cnt=0
    r_path=np.zeros(cnt_cities+1,dtype=np.int32)

    for i in range(0,cnt_cities):
        
        if path[i]==0:
            checker=1
        if checker==1:
            r_path[idx_cnt]=path[i]
            idx_cnt=idx_cnt+1
            
    if idx_cnt!=cnt_cities:
        for i in range(0,cnt_cities-idx_cnt):
            r_path[idx_cnt+i]=path[i]

    return path_map, r_path

def initialize_random(coord_list, first_idx):
    cnt_cities = len(coord_list)
    path = np.zeros(cnt_cities + 1, dtype=np.int32)

    path[0], path[-1] = first_idx, first_idx
    # Euclidean distance map between cities
    path_map = euclidean_distances(coord_list, coord_list)

    # city indices without first city index
    cities_tovisit = np.delete(np.arange(cnt_cities), first_idx)
    cities_random = np.random.permutation(cities_tovisit)
    path[1:-1] = cities_random

    return path_map, path

def path_cost(path_map, path):
    # The array of cost between cities in the path
    cnt_cities = path_map.shape[0]
    cost_arr = np.zeros(cnt_cities)
    for i in range(cnt_cities):
        cost_arr[i] = path_map[path[i], path[i+1]]

    return cost_arr

# Step 3. Searching a path

## Algorithm 4. Genetic Algorithm 

In [ ]:
def two_opt_swap(path_map, path, iterations, coord_list):
    cnt_cities = path_map.shape[0]
    # Save the best path

    cost_arr = path_cost(path_map, path)
    best_path = path.copy()
    best_cost = cost_arr.sum()
    
    for i in range(iterations):
        curr_path = best_path.copy()
        # Select two indices of flip points
        sel_idx = np.sort(np.random.choice(np.arange(1, cnt_cities + 1), 2))

        # Path Flip and update cost array
        curr_path[sel_idx[0]:sel_idx[1]] = np.flip(curr_path[sel_idx[0]: sel_idx[1]])
        cost_arr = path_cost(path_map, curr_path)

        # Compare to the best path
        curr_cost = cost_arr.sum()
        if curr_cost < best_cost:
            best_path = curr_path
            best_cost = curr_cost
    
    temperature = TEMPERATURE
    while temperature > TEMP_LIMIT:
        curr_path = best_path.copy()
        # Select two indices of flip points
        sel_idx = np.sort(np.random.choice(np.arange(1, cnt_cities + 1), 2))

        # Path Flip and update cost array
        curr_path[sel_idx[0]:sel_idx[1]] = np.flip(curr_path[sel_idx[0]: sel_idx[1]])
        cost_arr = path_cost(path_map, curr_path)
        curr_cost = cost_arr.sum()

        if curr_cost <= best_cost:
            best_path, best_cost = curr_path, curr_cost
        else:
            prob = 1 / np.exp((curr_cost - best_cost) / float(temperature))
            if prob > np.random.rand(1):
                best_path, best_cost = curr_path, curr_cost
        temperature = temperature * COOLING_RATIO 
    return best_path, best_cost

In [ ]:
def sa(path_map, path, coord_list):
    best_path, best_cost = path.copy() , path_cost(path_map, path).sum()

    for i in range(MAX_EVALUATION):
        curr_path = best_path.copy()
        new_path, new_cost = two_opt_swap(path_map, curr_path, SUB_ITERATIONS, coord_list)

        if new_cost < best_cost:
            best_path, best_cost = new_path, new_cost
            
    return best_path, best_cost

In [ ]:
def initialization(coord_list):
    cnt_cities = len(coord_list)
    path_pool = np.zeros((POOL_SIZE, cnt_cities + 1), dtype=np.int32)
    pool_cost = np.zeros(POOL_SIZE)
    
    path_map, path_pool[0, :] = initialize_greedy(coord_list, FIRST_IDX)
    pool_cost[0] = path_cost(path_map, path_pool[0, :]).sum()

    print('Path {} is initialized'.format(0))
    
    _, path_pool[1, :] = initialize_greedy_improve(coord_list, FIRST_IDX)
    pool_cost[1] = path_cost(path_map, path_pool[1, :]).sum()
    print('Path {} is initialized'.format(1))

    for i in range(2, POOL_SIZE):
        _, path_pool[i, :] = initialize_random(coord_list, FIRST_IDX)
        path_pool[i, :], pool_cost[i] = sa(path_map, path_pool[i, :], coord_list)
        print('Path {} is initialized'.format(i))
    
    return path_pool, pool_cost, path_map

In [ ]:
def selection(pool_cost,i, TOURNAMENT_SIZE, sel_size=2):
    # tournament selection
    if i%3==0:
        sel_idx = np.random.choice(POOL_SIZE, TOURNAMENT_SIZE, replace=False)
        sel_cost = pool_cost[sel_idx]
        best_idx = sel_idx[np.argsort(sel_cost)][:sel_size]
        
    elif i%3==1:
        idx_sort=np.argsort(pool_cost)
        sel_idx=idx_sort
        sel_cost = pool_cost[sel_idx]
        best_idx = sel_idx[np.argsort(sel_cost)][:sel_size]
    else:
        idx_sort=np.argsort(pool_cost)
        sel_idx=np.zeros(2,dtype=np.int32)
        sel_idx[0]=idx_sort[0]
        sel_idx[1]=np.random.choice(POOL_SIZE,1)
        while sel_idx[0]==sel_idx[1]:
            sel_idx[1]=np.random.choice(POOL_SIZE,1)
        best_idx=sel_idx
    return best_idx

In [ ]:
# pmx crossover
def crossover(path1, path2):
    cnt_cities = len(path1) - 1
    # Select two indices of crossover points
    sel_idx = np.sort(np.random.choice(np.arange(1, cnt_cities), 2))
    # Initialize child path
    child_path = np.zeros(cnt_cities + 1, dtype=np.int32)
    child_path[0], child_path[-1] = -1, -1
    # Copy the path between crossover points
    child_path[sel_idx[0]:sel_idx[1]] = path1[sel_idx[0]:sel_idx[1]]
    # Copy the rest of the path from path2
    path2_idx = np.where(np.isin(path2, child_path) == False)[0]
    child_path[np.where(child_path == 0)[0]] = path2[path2_idx]
    child_path[0], child_path[-1] = FIRST_IDX, FIRST_IDX

    return child_path


In [ ]:
# swap mutation
def mutation(path):
    cnt_cities = len(path)
    child_path = path.copy()
    # Select two indices of mutation points
    sel_idx = np.sort(np.random.choice(np.arange(1, cnt_cities), 2))
    # Swap the path between mutation points
    child_path[sel_idx[0]:sel_idx[1]] = np.flip(child_path[sel_idx[0]:sel_idx[1]])

    return child_path 


In [ ]:
# genetic algorithm
def ga(coord_list):
    best_cost = np.Inf
    print('Start Genetic Algorithm')
    print('Initialize the population')
    path_pool, pool_cost, path_map = initialization(coord_list)
    print('Start the evolution')
    for i in range(MAX_ITERATION):
        if (i+1) % 1000 == 0:
            print('Iteration {}'.format(i + 1))
        # selection
        sel_idx = selection(pool_cost,i, TOURNAMENT_SIZE)
        # crossover
        child_crx_1 = crossover(path_pool[sel_idx[0]], path_pool[sel_idx[1]])
        cost_crx_1 = path_cost(path_map, child_crx_1).sum()
        child_crx_2 = crossover(path_pool[sel_idx[1]], path_pool[sel_idx[0]])
        cost_crx_2 = path_cost(path_map, child_crx_2).sum()
        if cost_crx_1>cost_crx_2:
            child_crx=child_crx_2
            cost_crx=cost_crx_2
        else:
            child_crx=child_crx_1
            cost_crx=cost_crx_1
        
        # mutation
        sel_idx = selection(pool_cost,i, TOURNAMENT_SIZE, sel_size=1)
        child_mut = mutation(path_pool[sel_idx[0]])
        cost_mut = path_cost(path_map, child_mut).sum()
        # replace
        sort_idx = np.argsort(pool_cost)

        path_pool[sort_idx[-1]], pool_cost[sort_idx[-1]] = child_crx, cost_crx 
        path_pool[sort_idx[-2]], pool_cost[sort_idx[-2]] = child_mut, cost_mut 

        cur_idx = np.argmin(pool_cost)
        cur_path = path_pool[cur_idx]
        cur_cost = pool_cost[cur_idx]

        if best_cost > cur_cost:
            best_cost = cur_cost
            if PLOT_MODE:
                plt.close()
                figure, ax = plt.subplots()
                plt.scatter(coord_list[:, 0], coord_list[:, 1], c='red', s=10)
                plt.title('City Route: Iteration {}'.format(i + 1))
                coord_path = coord_list
                coord_path = np.append(coord_path, coord_path[FIRST_IDX, :].reshape(1, 2), axis=0)
                coord_path[:, :] = coord_path[cur_path, :]
                lines, = ax.plot(coord_path[:, 0], coord_path[:, 1], 'k--')
                figure.canvas.draw()
                figure.canvas.flush_events()
                plt.show()

    best_idx = np.argmin(pool_cost)
    return path_pool[best_idx], pool_cost[best_idx]

# Main

In [ ]:
# Step 1
try:
    coord_list = fileloader()
except Exception as e:
    print('예외 발생', e)
    sys.exit()

start_time = time.time()

best_path, best_cost = ga(coord_list)

print('Execution Time: ' + str(time.time() - start_time))
print('Path: ' + str(best_path.tolist()))
print('Cost: ' + str(best_cost))

In [ ]:
#greedy 기존
#Execution Time: 9.452116966247559
#Cost: 647622.3022318124

#greedy 개선
#Execution Time: 17.53960084915161
#Cost: 652969.7942243333

#SA 기존
#Execution Time: 38.89421820640564
#Cost: 647608.1235518232

#SA 개선
#Execution Time: 49.45330834388733
#Cost: 652969.7942243333

#GA 기존
#Execution Time: 34.25278615951538
#Cost: 647622.3022318124

#GA 개선
#Execution Time: 58.39043045043945
#Cost: 647622.3022318124

#비교
#기존의 data들에서는 greedy algorithm에서 미세하게 개선효과를 볼 수 있었으며, 이를 통해 전반적인 cost에서 이점을 볼 수 있었습니다.
#하지만 현재 매우 많은 양의 data를 처리하는 과정에서 기존에 생각한 greedy 알고리즘의 개선안이 효과적으로 동작하지 못하였으며,
#때문에 개선효과를 얻지 못하고 오히려 성능이 떨어지게 되었습니다.
#cost가 대략 2%정도 증가하였으며, initialization도 기존보다 연산과정이 많은데, 많은 양의 data를 처리하면서 execution time도 함께 증가하였습니다.

#문제점

#greedy algorithm의 값이 시작지점에 관계없이 대체로 고정되며, 때문에 시작지점을 적절히 선택하여 시작지점과 마지막 도착지점 간의 cost가 작다면
#initial value에서 우월한 지점을 선점하고 갈 수 있을 것이라 생각하였지만, 생각과는 달리 전체 cost가 더 증가하였습니다.
#또한 정렬 후 greedy를 진행하기 때문에 초기화 과정에서 정렬 하는 만큼 시간 소요가 더 됩니다.
#최단경로가 가장 긴 지점부터 greedy를 진행한다면 효과적으로 동작할 것으로 생각했지만, 이부분 또한 기존의 greedy 코드를 재사용해 
#greedy algorithm이 TSP 문제에서 가지는 근본적인 문제점을 해소하지는 못한다는 것을 깨닳았습니다. 
#(시작지점과 끝지점만 달라지며, 이또한 기존보다 이점을 얻지 못할 수 있다.)
#greedy의 문제점 - 시작 지점에서 이점을 보지만, 가면 갈수록 cost가 높은 link가 남게됨.
#SA 문제점 - T를 줄이는 함수 설계가 잘못되어 변화가 단조롭다고 생각됨. 안정적으로 개선효과를 보기 위해선 T를 줄이는 좋은 방법이 필요하다고 생각됨
#cost가 가장 큰 지점을 선택해 교체한다면 이점을 많이 볼 수 있을 것이라 생각하였는데, 이부분에 대한 재고도 필요하다고 생각됨.
#가장 큰 cost와 그 다음 cost => 서로 이웃해 있을 가능성 높음, 원하는 개선효과 얻기 힘들 것으로 판단
#가장 큰 cost와 가장 작은 cost 교체 => 이 둘도 이웃해 있을 가능성 있으며, 기존에 생각한 greedy의 문제점(시작지점과 끝지점의 거리가 멀다)이 더욱
#부각되어 문제 지점이 2배로 증가 할 수도 있다고 생각됨.
#때문에 가장 큰 cost와 중간 cost 혹은 random하게 선택하여 교체하였는데, 이또한 2-opt일 경우 미세한 향상효과만 얻게 될 확률이 높다고 생각됨.
#GA 문제점 - initial에서 많은 시간을 소비해 iteration 가능 횟수가 더더욱 제한됨. greedy와 SA 개선시 iteration이나 pool 등의 변수 제약에서
#조금 더 여유로워 질 수 있을 것으로 생각됨.

#해결방안

#greedy - path를 2개로 나누어 출발지->임시 도착지, 임시도착지->출발지로 하는 2개의 optimal값의 합으로 구함
#출발지와 임시 도착지는 link들 중에서 제일 cost가 큰 값을 선정하여 한점을 출발지, 다른 한점은 도착지
#두 지점을 연결하는 선을 중심으로 path를 나누어 우선 진행.

#SA - 가장 높은 cost를 선택하게 된다면, 2-opt일 경우 그 효과가 미비할수도 있다.
#가장 높은 지점과 중간지점 교체 -> 두 지점간의 득실관계 비교 -> 더 덜 줄었거나 증가한 지점과 임의의 한 지점(혹은 선택 지점)간 2-opt 한번 더 진행
#=> 3-opt 혹은 4-opt 등의 방법 생각

#T 줄이기 => cost가 감소 vs 변화x 에서, cost가 감소는 아직 local optima에 도달하지 못했을 가능성이 높음.
#bad case를 accept했다면 기존보다 cost를 조금 더 줄여서 탈출 할 수 있다면 좋겠다.
#그렇다면 bad case를 accept 했을 때 cooling ratio를 증가시켜 routine을 더 많이 겪게 한다면?
#local optima에 빠지게 될 확률은 언제나 있으므로, cooling ratio를 유동적으로 동작 가능하게 한다면?
#예를들어 bad case가 1번도 없었다 => T가 특정 수치 이상 내려간다면 cooling ratio를 높여 local optima 탈출 기회 높임
#bad case에 진입한다면 cost가 특정 횟수 이상 감소하기 전까지 cooling ratio를 높이거나 T값을 유지시켜 원하는 횟수까지는 감소하도록 설계할 수도 있음.

#GA - pool size와 토너먼트의 수 변경? 혹은 selection이나 2-opt하는 지점 선택에서 더 좋은 선택지가 있을 것이라 생각됨.
#selection - 기존에는 가장 좋은 cost를 가지는 경우를 하나 선택하는 경우를 많이 선택하였음.
#가장 좋은 cost를 선택하는 것은 좋은 것 같음. 하지만 생각해보니 현재 initial된 것중 가장 좋은 것과 random(SA된 이후)하게 선택된 것들 중 2개를 선택하는데,
#이를 iteration하는 동안 계속 같은 값을 이용해 계산하는 경우가 많았음(random하게 되는 경우를 제외하고)
#전회차에서 가장 작은 녀석 + random 혹은 그 다음으로 작은 녀석을 지속적으로 선택, 회차가 진행될수록 우월하다고 판단되는 case가 증가하도록 구현하는 것이 좋을 것 같음.
#두개의 case가 선택 된다면 교체되는 위치 또한 random 선택인데, 이를 더 효과적으로 수정할 수 있다면?
#예를 들어 가장 cost가 낮은 case에서 link cost가 가장 큰 node를 하나는 선택하도록 한다면?
#SA의 경우와 마찬가지로 2-opt일 경우 미비할 수 있음. 3-opt 혹은 4-opt를 통해 더 개선 할 여지 있을 것으로 판단됨.